<a href="https://colab.research.google.com/github/neversaynever0502/ai_rap_song_creation_by_chatgpt_whisper_hackathon/blob/main/AI_Rap_Song_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube_dl pytube
!pip install -U openai-whisper
!pip install openai
!pip install tqdm
!pip install pydub
!pip install gtts
!pip install librosa
!pip install picotts
!pip install moviepy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 KB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=c9b8a4b946b175b49060ddbe3262cf1849d3de0d

In [7]:
import os
import tempfile
from pytube import YouTube
from moviepy.editor import *
import requests
import whisper
import openai
from IPython.display import Markdown, display, Image
import string
import codecs
import io
from gtts import gTTS
from pydub import AudioSegment
from pydub.silence import detect_silence
from pydub.playback import play
import librosa
from IPython.display import Audio
import getpass



def download_youtube_video_tonewName(url):
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True).first()
    filename = stream.download()
    new_filename = "music.mp3"
    os.rename(filename, new_filename)

    # 轉換音訊編解碼器
    sound = AudioSegment.from_file(new_filename)
    sound.export(new_filename, format="mp3", bitrate="192k")

    return new_filename    
    

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True).first()
    return stream.download()

def extract_audio_from_video(video_path):
    with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_audio:
      
        audioclip = AudioFileClip(video_path)

        allAudio = []
      # 切割音訊
        duration = audioclip.duration
        start_time = 0 # 起始時間，單位為秒
        end_time = 660 # 結束時間，單位為秒
        count = 1
        while start_time < duration:
            if end_time > duration:
                end_time = duration
            # 切割音訊
            new_audioclip = audioclip.subclip(start_time, end_time)

            name = temp_audio.name
            # 使用 replace() 函数删除空格
            name = name.replace(" ", "")

            # 使用 translate() 函数删除标点符号
            name = name.translate(str.maketrans("", "", string.punctuation))

            # 儲存音訊
            output_path = "{}_{}.mp3".format(temp_audio.name,count)
            new_audioclip.write_audiofile(output_path)

            # 更新時間
            start_time = end_time
            end_time += 600
            count += 1
            # audioclip.write_audiofile(temp_audio.name)
            allAudio.append(output_path)

           
        return allAudio

def transcribe_audio(whisper_api_key, audio_path):
    # headers = {
    #     "Authorization": f"Bearer {whisper_api_key}",
    #     "Content-Type": "audio/mpeg"
    # }
    with open(audio_path, "rb") as audio_file:
        openai.api_key = whisper_api_key
        audio_file = open(audio_path, "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        print('transcript:',transcript.text)
        return transcript.text

def transcribe_audio_whisper(whisper_api_key, audio_path):
  model = whisper.load_model("base")

  result = model.transcribe(audio_path)
  result["text"]

def useGPT(whisper_api_key,text,codeType='python'):
    openai.api_key = whisper_api_key

    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      max_tokens=1028,
      temperature=0.5,
      messages=[
            # {"role": "user", "content": "我叫做 oxxo"},
            # {"role": "assistant", "content": "原來你是 oxxo 呀"},
            # {"role": "user", "content": "請問我叫什麼名字？"}
            # {"role":"user","content":'以下是一個教學{}的文字稿，幫我總結成幾點:\n'.format(codeType)+text}
          {"role":"user","content":'please help me summarize to write it as a {} rap song lyric:\n'.format(codeType)+text}

      ]
    )
    return response.choices[0].message.content

# def writeCodeGPT(whisper_api_key,text,codeType='python'):

def useDALLE(whisper_api_key,prompt):
    openai.api_key = whisper_api_key
    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="512x512",
    )
    return response["data"][0]["url"]

def imageShow(url):
    # url = useDALLE(whisper_api_key,prompt)
    # print(url)
    response = requests.get(url)
    img = Image(response.content)
    display(img)

def seconds_to_srt_time(seconds):
    """
    將秒數轉換為 SRT 字幕檔案中的時間格式
    """
    hours = int(seconds // 3600)
    minutes = int((seconds // 60) % 60)
    seconds = int(seconds % 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

image_url = ''

def main():
    # whisper_api_key = input("OpenAI apikey: ")
    whisper_api_key = getpass.getpass("OpenAI apikey: ")
    
    youtube_url = input("YouTube Url：")
    instrumentals = input("Youtube instrumentals Url: ")
    codeType = input("lyrics type：(optional)")

    video_path = download_youtube_video(youtube_url)
    audio_path_all = extract_audio_from_video(video_path)

    print('audio_path_all:',audio_path_all)

    transcriptCombine = ''
    for i in audio_path_all:
      transcript = transcribe_audio(whisper_api_key, i)

      print("video content：")
      print(transcript)
      transcriptCombine = transcriptCombine + transcript
      # os.remove(video_path)
      os.remove(i)
      # 将字符串编码为 bytes 类型

    print('whole video content：')
    print(transcriptCombine)

    # 將字串切割成每2000個字為一組
    # text = transcriptCombine
    # chunks = []
    # while len(text) > 0:
    #   if len(text) > 1000:
    #       chunk = text[:1000]
    #       text = text[1000:]
    #   else:
    #       chunk = text
    #       text = ""
    #   chunks.append(chunk)
    # # 顯示切割後的結果
    # for chunk in chunks:
    #     print('chunk:',chunk)
          # gptAnswer = useGPT(whisper_api_key, chunk ,codeType)

    gptAnswer = useGPT(whisper_api_key, transcriptCombine ,codeType)
    print('gptAnswer:')
    print(gptAnswer)

    gptAnswerArr = gptAnswer.split('\n')


    sections = ["Verse", "Chorus", "Outro",":","1","2","3",'4']

    lyrics_arr = []

    for i in gptAnswerArr:
      for section in sections:
          if section in i:
              i = i.replace(section, "")
      if i.strip() != '':
        lyrics_arr.append(i.strip())
    
    # download lyrics
    download_youtube_video_tonewName(instrumentals)


    # 載入音樂檔案，sr是採樣率，None代表採用原始採樣率
    y, sr = librosa.load('music.mp3', sr=None)

    # 計算節拍和節奏
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

    # 輸出節拍和節奏
    print('Tempo:', tempo)
    # print('Beat frames:', beat_frames)

    # 將節奏轉換為時間
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    print('Beat times:', beat_times[0:20])

    count = 0
    everyBeats = 8
    output = AudioSegment.silent(duration=100000)
    over_music_time = 0

    timeline = []

    for i, time in enumerate(beat_times):
        if i % everyBeats == 0:
          if count<len(lyrics_arr):
            start = int(time * 1000)-700
            end = int(beat_times[i+everyBeats] * 1000)-700
            text = lyrics_arr[count]
            count = count+1
            if text != '':
              print('timeline - '+str(start)+': '+text)
              tts = gTTS(text,lang='en', tld='com.au')
              tts.save('temp.mp3')
              segment = AudioSegment.from_file('temp.mp3')
              # segment = segment.speedup(playback_speed=1)
              # output = output.overlay(segment, position=start)

              # 切割原始音訊，只保留需要的部分
              sound_slice = output[start:end]

              # 疊加音訊段落
              result = sound_slice.overlay(segment)

              # 將疊加後的聲音段落插入到原始音訊中
              output = output[:start] + result + output[end:]

              over_music_time = end
              timeline.append((start,text))



    music = AudioSegment.from_file("music.mp3", format="mp3")
    mixed = music.overlay(output)

    mixed = mixed[:over_music_time+5000]


    mixed.export("output.mp3", format="mp3")
    global image_url
    image_url = useDALLE(whisper_api_key,transcriptCombine)
    # save to srt
    srt = ""
    for i, (timestamp, text) in enumerate(timeline):
        start_time = seconds_to_srt_time(timestamp / 1000)
        end_time = seconds_to_srt_time(timeline[i+1][0] / 1000) if i < len(timeline) - 1 else seconds_to_srt_time(timestamp / 1000 + 5)
        srt += f"{i+1}\n{start_time} --> {end_time}\n{text}\n\n"

    with open("output.srt", "w") as f:
        f.write(srt)




# Example YouTube Url: https://www.youtube.com/watch?v=LlqqM1a1GQI
# Example Youtube instrumentals Url: https://www.youtube.com/watch?v=6MFo-VvDohE

main()
Audio("output.mp3")





OpenAI apikey: ··········
YouTube Url：https://www.youtube.com/watch?v=LlqqM1a1GQI
Youtube instrumentals Url: https://www.youtube.com/watch?v=6MFo-VvDohE
lyrics type：(optional)
MoviePy - Writing audio in /tmp/tmpbyh6p8e5.mp3_1.mp3


MoviePy - Done.
audio_path_all: ['/tmp/tmpbyh6p8e5.mp3_1.mp3']
transcript: A hackathon is an event where people gather to solve a problem. It can be technical, educational, social, artistic or related to any field. This hackathon is about an organiser who has a problem and participants who are trying to solve the problem. These participants gather in teams made up of different skill sets to try and find a solution to the problem. At the end of the hackathon, the organiser selects the best solution and one team wins a prize. An incubation process between the winning team and the organiser is often launched at this point to transform the concept into a minimum viable product.
video content：
A hackathon is an event where people gather to solve a problem. It can be technical, educational, social, artistic or related to any field. This hackathon is about an organiser who has a problem and participants who are trying to solve the problem. These participants gather in teams made up of differe

In [ ]:
Image(url=image_url)